# Setting Up

In [ ]:
!nvidia-smi

In [ ]:
!pip install pandas==1.3.3 numpy==1.21.4 scikit-learn==0.24.2 matplotlib==3.4.3 seaborn==0.11.2 torch torchvision==0.10.1 torchtext==0.10.1


In [3]:
!pip install -q gradio torch

In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import joblib

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
%cd /content/drive/My Drive/Medicine

/content/drive/My Drive/Medicine


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#Data

In [166]:
# Load Dataset
df = pd.read_csv("drug_details.csv")

In [167]:
df.head(5)

,name,description,uses,dosage,side_effects
0,abilify,abilify\nabilify\nWhat is Abilify?\nIf you hav...,What is Abilify used for?\nIf you have a certa...,What is Abilify’s dosage?\nYour doctor will re...,What are Abilify’s side effects?\nLike most dr...
1,actemra,actemra\nactemra\nWhat is Actemra?\nIf you hav...,What is Actemra used for?\nActemra is used to ...,What is Actemra’s dosage?\nBelow are commonly ...,What are Actemra’s side effects?\nLike most dr...
2,actos,NaN,NaN,What is the dosage for Actos?\nYour doctor wil...,What are the side effects of Actos?\nLike most...
3,afinitor,afinitor\nafinitor\nWhat is Afinitor?\nIf you ...,What is Afinitor used for?\nAfinitor and Afini...,What is Afinitor’s dosage?\nYour doctor will r...,What are Afinitor’s side effects?\nLike most d...
4,aimovig,NaN,What is Aimovig used for?\nIf you have migrain...,What is Aimovig’s dosage?\nYour doctor will re...,What are Aimovig’s side effects?\nLike most dr...


In [168]:
df.shape


(6032, 5)

In [169]:
# Iterate through columns and perform text cleaning
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column contains text
        df[column] = df[column].str.lower()

In [170]:
#Replace NaN values in "description," "uses," "dosage," and "side_effects" columns with values from the "name" column
df['description'].fillna(df['name'], inplace=True)
df['uses'].fillna(df['name'], inplace=True)
df['dosage'].fillna(df['name'], inplace=True)
df['side_effects'].fillna(df['name'], inplace=True)

In [171]:
df.shape

(6032, 5)

In [172]:
# Drop rows with missing values
df = df.dropna().reset_index(drop=True)

In [173]:
df.shape

(5980, 5)

In [174]:
df.head()

,name,description,uses,dosage,side_effects
0,abilify,abilify\nabilify\nwhat is abilify?\nif you hav...,what is abilify used for?\nif you have a certa...,what is abilify’s dosage?\nyour doctor will re...,what are abilify’s side effects?\nlike most dr...
1,actemra,actemra\nactemra\nwhat is actemra?\nif you hav...,what is actemra used for?\nactemra is used to ...,what is actemra’s dosage?\nbelow are commonly ...,what are actemra’s side effects?\nlike most dr...
2,actos,actos,actos,what is the dosage for actos?\nyour doctor wil...,what are the side effects of actos?\nlike most...
3,afinitor,afinitor\nafinitor\nwhat is afinitor?\nif you ...,what is afinitor used for?\nafinitor and afini...,what is afinitor’s dosage?\nyour doctor will r...,what are afinitor’s side effects?\nlike most d...
4,aimovig,aimovig,what is aimovig used for?\nif you have migrain...,what is aimovig’s dosage?\nyour doctor will re...,what are aimovig’s side effects?\nlike most dr...


In [175]:
# Features & Labels
X_features = df['description']
y_labels = df[['name', 'uses', 'dosage', 'side_effects']]



In [176]:
from joblib import dump
from sklearn.preprocessing import LabelEncoder

In [177]:
df['name'].value_counts()

oriahnn                      52
abilify                      26
prolia                       26
qelbree                      26
quetiapine-er-oral-tablet    26
                             ..
imitrex                      26
ingrezza                     26
ivermectin                   26
injectafer                   26
xubsolv                      26
Name: name, Length: 229, dtype: int64

In [178]:
df['description'].value_counts()

oriahnn                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [179]:
df['uses'].value_counts()

what is oriahnn used for?\nif you have heavy periods, your doctor may talk with you about oriahnn.\nit’s a prescription drug that’s used to treat heavy menstrual bleeding caused by fibroids in the uterus. (fibroids are noncancerous growths.) this drug is prescribed for use in adult females* who haven’t yet gone through menopause.\nfibroids can cause you to have heavy bleeding with your periods. it’s not known for sure why this happens.\noriahnn works to lower certain hormone levels in your body. this way, the lining inside your uterus doesn’t grow as thick as usual. this leads to a lighter flow during your periods.\nit’s important to note that it’s not recommended to take oriahnn for longer than 24 months. this is because the drug may cause bone loss that can’t be reversed. the risk of bone loss with oriahnn is higher if you take the drug for more than 24 months and if you have other risk factors, such as:\nhaving a family history of bone loss\nsmoking\nhaving chronic (long-lasting) ki

In [180]:
df['dosage'].value_counts()

what is oriahnn’s dosage?\nyour doctor will recommend the dosage of oriahnn that’s right for you. below is the commonly used dosage, but always take the dosage your doctor prescribes.\nforms and strengths\noriahnn comes as two oral capsules. you’ll take one type of capsule in the morning and the other at night. the capsules are described in the table below.\nform colors active ingredients* and strengths\noriahnn morning capsule white and yellow • 300 milligrams (mg) elagolix,\n• 1 mg estradiol, and\n• 0.5 mg norethindrone acetate\noriahnn evening capsule white and blue • 300 mg elagolix\n* active ingredients are what makes a drug work.\nrecommended dosage\noriahnn is usually taken twice per day, once in the morning and once in the evening.\nquestions about taking oriahnn\nhere’s a list of common questions related to taking oriahnn.\nwhat if i miss a dose of oriahnn? if you miss a dose of oriahnn, either in the morning or evening, take it as soon as you remember as long as it’s within 4

In [181]:
df['side_effects'].value_counts()

what are oriahnn’s side effects?\nlike most drugs, oriahnn may cause mild or serious side effects. the lists below describe some of the more common side effects that oriahnn may cause. these lists don’t include all possible side effects.\nkeep in mind that side effects of a drug can depend on:\nyour age\nother health conditions you have\nother medications you may be taking\nyour doctor or pharmacist can tell you more about the potential side effects of oriahnn. they can also suggest ways to help reduce side effects.\nmild side effects\nhere’s a short list of some of the mild side effects that oriahnn can cause. to learn about other mild side effects, talk with your doctor or pharmacist, or read oriahnn’s prescribing information.\nmild side effects of oriahnn that have been reported include:\nhot flashes\nheadache\nfatigue (lack of energy)\nirregular periods\nweight gain\nmild side effects of many drugs may go away within a few days or a couple of weeks. but if they become bothersome, t

In [182]:
# Load Pkgs
from sklearn.multioutput import MultiOutputClassifier

In [183]:
# Features & Labels
Xfeatures = df['description']
ylabels = df[['name', 'uses', 'dosage', 'side_effects']]

In [184]:
# Split Data
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=7)

#Define pipeline

In [185]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# Define your pipeline
pipe_lr = Pipeline(steps=[
    ('cv', CountVectorizer()),
    ('lr_multi', MultiOutputClassifier(LogisticRegression()))
])


In [127]:
# Fit on Dataset
pipe_lr.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr_multi',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

In [129]:
# Accuracy Score
pipe_lr.score(x_test,y_test)

1.0

In [130]:
# Sample Prediction
print(x_test.iloc[0])
print("Actual Prediction:",y_test.iloc[0])

sunosi
sunosi
What is Sunosi?
If you have excessive daytime sleepiness, your doctor may recommend treatment with Sunosi.
Sunosi is a prescription drug used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.
In certain cases, Sunosi shouldn’t be prescribed for people with obstructive sleep apnea. To learn more, see the “What is Sunosi prescribed for?” section below.
Sunosi basics
The active ingredient in Sunosi is solriamfetol. An active ingredient is what makes a drug work.
Sunosi comes as tablets that you swallow. It’s not available as a generic drug.
Keep reading to learn more about Sunosi, including the drug’s side effects, uses, how it compares with similar medications, and more.
What is Sunosi used for?
Sunosi is used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.
What is narcolepsy?
Narcolepsy is a condition that causes abnormal sleep. This can lead to excessive daytime sleepiness an

In [131]:
ex1 = x_test.iloc[0]

In [132]:
ex1

'sunosi\nsunosi\nWhat is Sunosi?\nIf you have excessive daytime sleepiness, your doctor may recommend treatment with Sunosi.\nSunosi is a prescription drug used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.\nIn certain cases, Sunosi shouldn’t be prescribed for people with obstructive sleep apnea. To learn more, see the “What is Sunosi prescribed for?” section below.\nSunosi basics\nThe active ingredient in Sunosi is solriamfetol. An active ingredient is what makes a drug work.\nSunosi comes as tablets that you swallow. It’s not available as a generic drug.\nKeep reading to learn more about Sunosi, including the drug’s side effects, uses, how it compares with similar medications, and more.\nWhat is Sunosi used for?\nSunosi is used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.\nWhat is narcolepsy?\nNarcolepsy is a condition that causes abnormal sleep. This can lead to excessive daytime

In [133]:
pipe_lr.predict([ex1])

array([['sunosi',
        'What is Sunosi used for?\nSunosi is used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.\nWhat is narcolepsy?\nNarcolepsy is a condition that causes abnormal sleep. This can lead to excessive daytime sleepiness and “sleep attacks” (an overwhelming urge to fall asleep).\nSymptoms of narcolepsy can include:\nsignificant daytime sleepiness\ncataplexy (sudden loss of muscle tone that’s temporary)\nsleep paralysis\ntrouble falling or staying asleep\nWhat is obstructive sleep apnea?\nObstructive sleep apnea (OSA) is caused by narrowing of the airways during sleep, which cuts off breathing. This causes poor sleep quality, leading to excessive daytime sleepiness.\nSymptoms of OSA can include:\nloud snoring\ngasping, choking, or snorting during sleep\nmorning headache\nwaking repeatedly during the night\nforgetfulness\nNote: Sunosi is not used to treat the airway obstruction that causes OSA. Before taking Sunosi, peopl

In [134]:
# Prediction Prob
print(pipe_lr.classes_)
pipe_lr.predict_proba([ex1])

[array(['abilify', 'actemra', 'afinitor', 'alprazolam-ir-oral-tablet',
       'amlodipine-oral-tablet', 'amoxicillin', 'arimidex', 'aristada',
       'austedo', 'belsomra', 'blincyto', 'botox', 'briviact',
       'buprenorphine', 'bydureon-bcise', 'cabometyx', 'caplyta',
       'cosentyx', 'dupixent', 'enbrel', 'entyvio', 'epclusa',
       'epidiolex', 'erleada', 'eysuvis', 'fasenra', 'faslodex',
       'gemtesa', 'genvoya', 'gocovri', 'humulin-r', 'ilaris', 'imfinzi',
       'imitrex', 'ingrezza', 'invokana', 'januvia', 'kesimpta',
       'kineret', 'kymriah', 'lantus', 'leqvio', 'libtayo', 'lipitor',
       'lokelma', 'losartan-oral-tablet', 'lupron-depot', 'lybalvi',
       'lyumjev', 'metoprolol-oral-tablet', 'myrbetriq', 'narcan',
       'nexlizet', 'novolog', 'olumiant', 'opzelura', 'orencia',
       'ozempic', 'plaquenil', 'praluent', 'quviviq', 'repatha',
       'retevmo', 'rybrevant', 'sinemet', 'soliqua', 'spiriva',
       'spravato', 'stribild', 'sublocade', 'sunosi', 'taltz

[array([[4.26436161e-07, 2.48612839e-07, 4.35288802e-07, 8.95950068e-07,
         2.85442950e-07, 2.71586559e-06, 1.03446837e-06, 5.69151723e-06,
         7.79166943e-06, 2.71349471e-06, 2.28044151e-06, 2.41517527e-06,
         4.02920918e-06, 2.14191229e-07, 1.01581198e-06, 6.06532985e-06,
         3.42313446e-06, 5.00610122e-07, 2.85660709e-07, 5.27033862e-07,
         1.20464657e-06, 1.39127757e-06, 5.59849033e-06, 2.42484051e-06,
         3.29623864e-06, 5.96412669e-06, 6.16959331e-07, 3.19183040e-06,
         9.40551011e-07, 5.92424825e-06, 6.71937751e-07, 7.13032415e-07,
         3.85992328e-07, 8.06054169e-07, 1.83537566e-06, 9.99430669e-07,
         1.36226241e-06, 5.98048597e-07, 5.89355922e-06, 7.67680746e-07,
         4.91315648e-07, 1.43413794e-06, 1.64115187e-06, 2.63982225e-06,
         3.49493520e-06, 5.43442945e-07, 4.95062306e-07, 1.62259351e-06,
         1.33633470e-07, 3.60875920e-07, 3.46705980e-06, 1.62922532e-06,
         6.90724437e-06, 1.11482370e-06, 1.05924053

In [ ]:
# Save the trained model
model = 'Medicine_Library8.joblib'
dump(pipe_lr, model)
print("Model saved as", model)

In [136]:
from joblib import load

# Load the saved model
loaded_model = load(model)


In [137]:
loaded_model.predict([ex1])

array([['sunosi',
        'What is Sunosi used for?\nSunosi is used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.\nWhat is narcolepsy?\nNarcolepsy is a condition that causes abnormal sleep. This can lead to excessive daytime sleepiness and “sleep attacks” (an overwhelming urge to fall asleep).\nSymptoms of narcolepsy can include:\nsignificant daytime sleepiness\ncataplexy (sudden loss of muscle tone that’s temporary)\nsleep paralysis\ntrouble falling or staying asleep\nWhat is obstructive sleep apnea?\nObstructive sleep apnea (OSA) is caused by narrowing of the airways during sleep, which cuts off breathing. This causes poor sleep quality, leading to excessive daytime sleepiness.\nSymptoms of OSA can include:\nloud snoring\ngasping, choking, or snorting during sleep\nmorning headache\nwaking repeatedly during the night\nforgetfulness\nNote: Sunosi is not used to treat the airway obstruction that causes OSA. Before taking Sunosi, peopl

In [138]:
# Sample prediction
sample_input = [ex1]

# Predict probabilities for each class in each output
prediction_probs = loaded_model.predict_proba(sample_input)

# Print the predicted probabilities
print("Prediction Probabilities:")
print(prediction_probs)

Prediction Probabilities:
[array([[4.26436161e-07, 2.48612839e-07, 4.35288802e-07, 8.95950068e-07,
        2.85442950e-07, 2.71586559e-06, 1.03446837e-06, 5.69151723e-06,
        7.79166943e-06, 2.71349471e-06, 2.28044151e-06, 2.41517527e-06,
        4.02920918e-06, 2.14191229e-07, 1.01581198e-06, 6.06532985e-06,
        3.42313446e-06, 5.00610122e-07, 2.85660709e-07, 5.27033862e-07,
        1.20464657e-06, 1.39127757e-06, 5.59849033e-06, 2.42484051e-06,
        3.29623864e-06, 5.96412669e-06, 6.16959331e-07, 3.19183040e-06,
        9.40551011e-07, 5.92424825e-06, 6.71937751e-07, 7.13032415e-07,
        3.85992328e-07, 8.06054169e-07, 1.83537566e-06, 9.99430669e-07,
        1.36226241e-06, 5.98048597e-07, 5.89355922e-06, 7.67680746e-07,
        4.91315648e-07, 1.43413794e-06, 1.64115187e-06, 2.63982225e-06,
        3.49493520e-06, 5.43442945e-07, 4.95062306e-07, 1.62259351e-06,
        1.33633470e-07, 3.60875920e-07, 3.46705980e-06, 1.62922532e-06,
        6.90724437e-06, 1.11482370e-0

###Using ML Estimators KNN which supports multi-output

In [139]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

# Define your pipeline
pipe_knn = Pipeline(steps=[
    ('cv', CountVectorizer()),
    ('knn', KNeighborsClassifier(n_neighbors=4))
])


In [140]:
pipe_knn.fit(x_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('knn', KNeighborsClassifier(n_neighbors=4))])

In [ ]:
# Save the trained model
model = 'medicine_library8_knn.joblib'
dump(pipe_knn, model)
print("Model saved as", model)


In [142]:
from joblib import load

# Load the saved model
loaded_model = load(model)


In [143]:
loaded_model.predict([ex1])

array([['sunosi',
        'What is Sunosi used for?\nSunosi is used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.\nWhat is narcolepsy?\nNarcolepsy is a condition that causes abnormal sleep. This can lead to excessive daytime sleepiness and “sleep attacks” (an overwhelming urge to fall asleep).\nSymptoms of narcolepsy can include:\nsignificant daytime sleepiness\ncataplexy (sudden loss of muscle tone that’s temporary)\nsleep paralysis\ntrouble falling or staying asleep\nWhat is obstructive sleep apnea?\nObstructive sleep apnea (OSA) is caused by narrowing of the airways during sleep, which cuts off breathing. This causes poor sleep quality, leading to excessive daytime sleepiness.\nSymptoms of OSA can include:\nloud snoring\ngasping, choking, or snorting during sleep\nmorning headache\nwaking repeatedly during the night\nforgetfulness\nNote: Sunosi is not used to treat the airway obstruction that causes OSA. Before taking Sunosi, peopl

In [145]:
# Sample prediction
sample_input = [ex1]

# Predict probabilities for each class in each output
prediction_probs = loaded_model.predict_proba(sample_input)

# Print the predicted probabilities
print("Prediction Probabilities:")
print(prediction_probs)


Prediction Probabilities:
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [146]:
Pipeline(steps=[('cv', CountVectorizer()),
                ('knn', KNeighborsClassifier(n_neighbors=4))])

Pipeline(steps=[('cv', CountVectorizer()),
                ('knn', KNeighborsClassifier(n_neighbors=4))])

In [147]:
pipe_knn.predict([ex1])

array([['sunosi',
        'What is Sunosi used for?\nSunosi is used to treat excessive daytime sleepiness related to narcolepsy or obstructive sleep apnea in adults.\nWhat is narcolepsy?\nNarcolepsy is a condition that causes abnormal sleep. This can lead to excessive daytime sleepiness and “sleep attacks” (an overwhelming urge to fall asleep).\nSymptoms of narcolepsy can include:\nsignificant daytime sleepiness\ncataplexy (sudden loss of muscle tone that’s temporary)\nsleep paralysis\ntrouble falling or staying asleep\nWhat is obstructive sleep apnea?\nObstructive sleep apnea (OSA) is caused by narrowing of the airways during sleep, which cuts off breathing. This causes poor sleep quality, leading to excessive daytime sleepiness.\nSymptoms of OSA can include:\nloud snoring\ngasping, choking, or snorting during sleep\nmorning headache\nwaking repeatedly during the night\nforgetfulness\nNote: Sunosi is not used to treat the airway obstruction that causes OSA. Before taking Sunosi, peopl

#Inference

In [150]:
import gradio as gr
from joblib import load

# Load the saved model
#model_file = "drug_book_knn.joblib"
model_file = "Medicine_Library8.joblib"
loaded_model = load(model_file)

def predict_drug(text_input):
    # Perform prediction using the loaded model
    prediction = loaded_model.predict([text_input])[0]

    drug_name = prediction[0]
    drug_uses = prediction[1]
    drug_dosage = prediction[2]
    drug_side_effects = prediction[3]

    output_text = f"DRUG NAME:\n\n {drug_name}\n\n, DRUG USES:\n\n {drug_uses}, \n\nDRUG DOSAGE:\n\n {drug_dosage}\n\n, side effects:\n\n {drug_side_effects}"
    return output_text

# Create the interface
iface = gr.Interface(
    fn=predict_drug,
    inputs=gr.inputs.Textbox(lines=3, label="Enter drug description here: "),
    outputs=gr.outputs.Textbox(label="\n\nPredicted drug details\n\n")
)

# Launch the interface
iface.launch(share=True)





<ipython-input-150-b4155f7b01c5>:24: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=3, label="Enter drug description here: "),
<ipython-input-150-b4155f7b01c5>:24: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=3, label="Enter drug description here: "),
<ipython-input-150-b4155f7b01c5>:24: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=3, label="Enter drug description here: "),
<ipython-input-150-b4155f7b01c5>:25: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(label="\n\nPredicted drug details\n\n")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0bcdda8bc8c7525bc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
